<h1><center><font size="6">Customer Segmentation Using RFM</font></center></h1>

<h2><center><font size="4">Dataset used: Intuilize Customer transaction data</font></center></h2>



When it comes to finding out who your best customers are, the old RFM matrix principle is the best. RFM stands for Recency, Frequency and Monetary. It is a customer segmentation technique that uses past purchase behavior to divide customers into groups.

#### RFM Score Calculations

__RECENCY (R):__ Days since last purchase <br> 

__FREQUENCY (F):__ Total number of purchases<br> 

__MONETARY VALUE (M):__ Total money this customer spent <br> 


__Step 1:__ Calculate the RFM metrics for each customer. 
<img src="https://cdn-images-1.medium.com/max/1200/1*uqIFE9igGhmxadB_dLz-WQ.png" width="600"></img>

<br>


__Step 2:__ Add segment numbers to RFM table.


<img src ="https://cdn-images-1.medium.com/max/1200/1*tfHBghAMkF_dUcaQa72tpQ.png"></img> <br>

__Step 3:__ Sort according to the RFM scores from the best customers (score 111).


<img src = "https://cdn-images-1.medium.com/max/1200/1*MeXvFG4Ez0wStMDCXZyN-Q.png"></img> <br>

**Since RFM is based on user activity data, the first thing we need is data.**

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv("sales_data.csv")
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Quantity Shipped,Year,Invoice Date,UnitPrice,Ext-Price,CustomerID,Country,GM%,Shipper,COST,EXT COST,REGION,SALES P,OrderPeriod
0,0,568200,PADS,5.00,"2,017",2017-09-25 00:00:00+00:00,1.0,$5.00,16198,United Kingdom,0%,Delivery Truck,1.0,4.999763,SOUTH,GRACE J,2017-09
1,1,568375,BANK CHARGES,5.00,"2,017",2017-09-26 00:00:00+00:00,1.0,$5.00,13405,United Kingdom,100%,UPS Next day Air,0.0,0.000000,WEST,JEFF Y,2017-09
2,2,561226,PADS,5.00,"2,017",2017-07-26 00:00:00+00:00,1.0,$5.00,15618,United Kingdom,0%,Delivery Truck,1.0,4.999763,SOUTH,GRACE J,2017-07
3,3,2550193,PADS,7.08,"2,018",2018-06-08 00:00:00+00:00,1.0,$7.08,13952,United Kingdom,0%,UPS ground 2nd day,1.0,7.079664,CENTRAL,PETER T,2018-06
4,4,550193,PADS,5.00,"2,017",2017-04-15 00:00:00+00:00,1.0,$5.00,13952,United Kingdom,0%,UPS ground 2nd day,1.0,4.999763,CENTRAL,PETER T,2017-04


In [4]:
df['Quantity Shipped'] = round(df['Quantity Shipped'],0)

In [8]:
df = df.iloc[:,1:]

In [9]:
df1 = df

### Explore the data — validation and new variables

* Missing values in important columns;
* Customers’ distribution in each country;
* Unit price and Quantity should > 0;
* Invoice date should < today.


In [10]:
df1.Country.nunique()

38

In [11]:
df1.Country.unique()

array(['United Kingdom', 'EIRE', 'Germany', 'France', 'Israel', 'Austria',
       'Norway', 'Finland', 'Spain', 'Belgium', 'Switzerland', 'Portugal',
       'Netherlands', 'Singapore', 'Japan', 'Sweden', 'Malta', 'Greece',
       'Hong Kong', 'Unspecified', 'Cyprus', 'Canada',
       'United Arab Emirates', 'Australia', 'Italy', 'Channel Islands',
       'Denmark', 'USA', 'Poland', 'Iceland', 'Czech Republic', 'RSA',
       'European Community', 'Lebanon', 'Saudi Arabia', 'Lithuania',
       'Bahrain', 'Brazil'], dtype=object)

In [12]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3753
14,Germany,91
13,France,88
10,EIRE,31
31,Spain,30
3,Belgium,24
33,Switzerland,22
27,Portugal,18
19,Italy,13
12,Finland,11


More than 90% of the customers in the data are from the United Kingdom. There’s some research indicating that customer clusters vary by geography, so here I’ll restrict the data to the United Kingdom only.

In [13]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

#### Check whether there are missing values in each column.



In [14]:
df1.isnull().sum(axis = 0)

InvoiceNo             0
StockCode             0
Quantity Shipped      0
 Year                 0
Invoice Date        218
UnitPrice             0
 Ext-Price            0
CustomerID            0
Country               0
GM%                   0
Shipper               0
COST                  0
EXT COST              0
REGION                0
SALES P               0
OrderPeriod           0
dtype: int64

There are 218 missing values in the Invoice date column and we will remove these missing values.

In [15]:
df1 = df1[pd.notnull(df1['Invoice Date'])]

#### Check the minimum values in UnitPrice and Quantity columns.

In [17]:
print(df1['UnitPrice'].min())
print(df1['Quantity Shipped'].min())

#Remove the negative values in Quantity column.

df1 = df1[(df1['Quantity Shipped']>0)]
df1.shape
df1.info()

0.084
0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 513430 entries, 0 to 566308
Data columns (total 16 columns):
InvoiceNo           513430 non-null object
StockCode           513430 non-null object
Quantity Shipped    513430 non-null float64
 Year               513430 non-null object
Invoice Date        513430 non-null object
UnitPrice           513430 non-null float64
 Ext-Price          513430 non-null object
CustomerID          513430 non-null int64
Country             513430 non-null object
GM%                 513430 non-null object
Shipper             513430 non-null object
COST                513430 non-null float64
EXT COST            513430 non-null float64
REGION              513430 non-null object
SALES P             513430 non-null object
OrderPeriod         513430 non-null object
dtypes: float64(4), int64(1), object(11)
memory usage: 66.6+ MB


After cleaning up the data, we are now dealing with 513430 rows and 16 columns.

Check unique value for each column.

In [18]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  22673
StockCode :  3893
Quantity Shipped :  1796
 Year  :  3
Invoice Date :  656
UnitPrice :  6666
 Ext-Price  :  37077
CustomerID :  3753
Country :  1
GM% :  680
Shipper :  4
COST :  262
EXT COST :  180218
REGION :  4
SALES P :  6
OrderPeriod :  25


In [20]:
#Add a column for total price.

df1['TotalPrice'] = df1['Quantity Shipped'] * df1['UnitPrice']
#Find out the first and last order dates in the data.

print(df1['Invoice Date'].min())


print(df1['Invoice Date'].max())


#Since recency is calculated for a point in time, and the last invoice date is 2018–12–27, we will use 2018–12–28 to calculate recency.

import datetime as dt
NOW = dt.datetime(2018,12,28)
df1['Invoice Date'] = pd.to_datetime(df1['Invoice Date'])

2016-12-01 00:00:00+00:00
2018-12-27 00:00:00+00:00


### RFM Customer Segmentation
*RFM segmentation starts from here.*

Create an RFM table:

In [21]:
rfmTable = df1.groupby('CustomerID').agg({'Invoice Date': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})
rfmTable['Invoice Date'] = rfmTable['Invoice Date'].astype(int)
rfmTable.rename(columns={'Invoice Date': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

In [23]:
#Calculate RFM metrics for each customer

rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346,233,2,750591.752
12431,9,970,55260.048
12433,8,1149,54884.026
12471,13,2781,161093.816
12583,15,878,50140.206


In [25]:
rfmTable.to_csv("rfm_table.csv")

### Interpretation:

> * CustomerID 12346 has frequency: 2, monetary value: $750591.752 and recency: 233 days <br>

> * CustomerID 12431 has frequency: 970, monetary value: $55260.048 and recency: 9 days

            
**Let’s check the details of the first customer.**

In [26]:
first_customer = df1[df1['CustomerID']==12346]

In [27]:
first_customer

,InvoiceNo,StockCode,Quantity Shipped,Year,Invoice Date,UnitPrice,Ext-Price,CustomerID,Country,GM%,Shipper,COST,EXT COST,REGION,SALES P,OrderPeriod,TotalPrice
109624,C541433,23166,17.0,"2,017",2017-01-18,1.096,$18.25,12346,United Kingdom,-18%,UPS ground 2nd day,1.29,21.442803,WEST,JEFF Y,2017-01,18.632
147733,2541431,23166,664224.0,"2,018",2018-05-09,1.130,"$750,849.72",12346,United Kingdom,-14%,UPS Next day Air,1.29,855425.223500,WEST,JEFF Y,2018-05,750573.120


The first customer has shopped only twice, bought one product at a huge quantity(664224) after a period of 476 days since the first order at quantity(17)

### Split the metrics

**The easiest way to split metrics into segments is by using quartiles.**

<br>
> * This gives us a starting point for the detailed analysis.
<br> 
> * 4 segments are easy to understand and explain.

In [28]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

### Create a segmented RFM table


*The lowest recency, highest frequency and monetary amounts are our best customers.*


In [29]:
segmented_rfm = rfmTable
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
#Add segment numbers to the newly created segmented RFM table

segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()


,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346,233,2,750591.752,3,4,1
12431,9,970,55260.048,1,1,1
12433,8,1149,54884.026,1,1,1
12471,13,2781,161093.816,1,1,1
12583,15,878,50140.206,1,1,1


*RFM segments split the customer base into an imaginary 3D cube which is hard to visualize. However, we can sort it out.*
<br>

**Add a new column to combine RFM score: 111 is the highest score as we determined earlier.**





In [31]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346,233,2,750591.752,3,4,1,341
12431,9,970,55260.048,1,1,1,111
12433,8,1149,54884.026,1,1,1,111
12471,13,2781,161093.816,1,1,1,111
12583,15,878,50140.206,1,1,1,111


It is obvious that the first customer is not our best customer at all.

Who are the top 10 of our best customers!

In [37]:
segmented_rfm[segmented_rfm['RFMScore']!='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,Segment,Marketing,Description
CustomerID,,,,,,,,,,
12346,233,2,750591.752,3,4,1,341,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
13145,470,5,668760.665,4,4,1,441,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
15098,82,3,559935.844,2,4,1,241,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
17381,41,143,348845.050,2,1,1,211,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
16003,7,96,274443.410,1,2,1,121,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
17857,13,60,259654.372,1,2,1,121,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
14224,55,181,225192.546,2,1,1,211,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
17306,54,82,221815.991,2,2,1,221,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."
14096,388,3240,173672.056,3,1,1,311,Best Customers,"No price incentives, new products, and loyalty...","Bought most recently, most often, and spend th..."


In [84]:
segmented_rfm.RFMScore.unique()

array(['341', '111', '122', '211', '323', '244', '444', '344', '434',
       '112', '333', '144', '131', '243', '432', '232', '332', '423',
       '222', '443', '312', '231', '242', '121', '343', '442', '221',
       '322', '311', '334', '433', '133', '321', '212', '422', '142',
       '233', '441', '143', '132', '123', '342', '223', '331', '411',
       '412', '424', '234', '431', '313', '241', '324', '141', '134',
       '421', '113'], dtype=object)

In [44]:
segmented_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3753 entries, 12346 to 18287
Data columns (total 10 columns):
recency           3753 non-null int32
frequency         3753 non-null int64
monetary_value    3753 non-null float64
r_quartile        3753 non-null int64
f_quartile        3753 non-null int64
m_quartile        3753 non-null int64
RFMScore          3753 non-null object
Segment           3753 non-null object
Marketing         3753 non-null object
Description       3753 non-null object
dtypes: float64(1), int32(1), int64(4), object(4)
memory usage: 307.9+ KB


In [87]:
for i in segmented_rfm.index:
    a =  segmented_rfm.loc[i,'r_quartile']
    b = segmented_rfm.loc[i,'f_quartile']
    c = segmented_rfm.loc[i,'m_quartile']
    if ( a == 1) & (b == 1) & (c == 1):
        segmented_rfm.at[i,'Segment'] = 'Best Customers'
        segmented_rfm.at[i,'Marketing'] = 'Focus on loyalty programs and new product introductions. These customers have proven to have a higher willingness to pay, so do not use discount pricing to generate incremental sales. Instead, focus on value added offers through product recommendations based on previous purchases'
        segmented_rfm.at[i,'Description'] = 'Highly engaged customers who have bought the most recent, the most often, and generated the most revenue.'
    elif (a == 4) & (b == 4) & (c == 4):
        segmented_rfm.at[i,'Segment'] = 'Lost Cheap Customers'
        segmented_rfm.at[i,'Marketing'] = 'Do not spend too much trying to re-acquire'
        segmented_rfm.at[i,'Description'] = 'Last purchased long ago, purchased few, and spent little'
    elif (a == 4) & (b == 1) & (c == 1):
        segmented_rfm.at[i,'Segment'] = 'Lost Customers'
        segmented_rfm.at[i,'Marketing'] = 'Aggressive price incentives'
        segmented_rfm.at[i,'Description'] = 'Have not purchased for some time but purchased frequently and spend the most'
    elif ((a == 1) & (b == 4) & (c == 1))|((a == 1) & (b == 4) & (c == 2)):
        segmented_rfm.at[i,'Segment'] = 'High Spending New Customers'
        segmented_rfm.at[i,'Marketing'] = 'Aggressive price incentives'
        segmented_rfm.at[i,'Description'] = 'customers who transacted only once, but very recently and they spent a lot.'
    elif ((a == 1) & (b == 1) & (c == 3))|((a == 1) & (b == 1) & (c == 4)):
        segmented_rfm.at[i,'Segment'] = 'Lowest Spending Active Loyal Customers'
        segmented_rfm.at[i,'Marketing'] = 'Aggressive price incentives'
        segmented_rfm.at[i,'Description'] = 'they transacted recently and do so often, but spend the least.'
    elif ((a == 4) & (b == 1) & (c == 1))|((a == 4) & (b == 1) & (c == 2))|((a == 4) & (b == 2) & (c == 2))|((a == 4) & (b == 2) & (c == 1)):
        segmented_rfm.at[i,'Segment'] = 'Churned Best Customers'
        segmented_rfm.at[i,'Marketing'] = 'Make strategies to get them back and buy'
        segmented_rfm.at[i,'Description'] = 'they transacted frequently and spent a lot, but it’s been a long time since they’ve transacted'
    elif (a == 3) & (b ==1) & (c == 1):
        segmented_rfm.at[i,'Segment'] = 'Almost Lost'
        segmented_rfm.at[i,'Marketing'] = 'Aggressive price incentives'
        segmented_rfm.at[i,'Description'] = 'Have not purchased for some time but purchased frequently and spend the most'
    elif (c == 1)& (b == 1) & (a !=1) &(a!=4):
        segmented_rfm.at[i,'Segment'] = 'Loyal customers and big spenders'
        segmented_rfm.at[i,'Marketing'] = 'market expensive products to them and stay engaged with them to maintain their loyalty'
        segmented_rfm.at[i,'Description'] = 'Buy most frequently and spends the most'
    elif (b ==1) & (c != 1) & (a!=1):
        segmented_rfm.at[i,'Segment'] = 'Other Loyal Customers'
        segmented_rfm.at[i,'Marketing'] = 'Maintain their loyalty by various adapted programs'
        segmented_rfm.at[i,'Description'] = 'buys most frequently but not the big spenders'
    elif (c==1) & (b!= 1) & (a!=1):
        segmented_rfm.at[i,'Segment'] = 'Other big spenders'
        segmented_rfm.at[i,'Marketing'] = 'market expensive products to them'
        segmented_rfm.at[i,'Description'] = 'spends the most but not frequently'
    else:
        segmented_rfm.at[i,'Segment'] = 'Others'
        segmented_rfm.at[i,'Marketing'] = 'Depending on the closeness to other scores'
        segmented_rfm.at[i,'Description'] = 'can be described on the basis of its proximity to other declared rfm scores'
        

In [76]:
segmented_rfm

array(['Other big spenders', 'Best Customers', 'Others',
       'Loyal customers and big spenders', 'Lost Cheap Customers',
       'Other Loyal Customers', 'Almost Lost', 'Lost Customers'],
      dtype=object)

In [88]:
segmented_rfm.groupby('Segment').count()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,Marketing,Description
Segment,,,,,,,,,
Almost Lost,44,44,44,44,44,44,44,44,44
Best Customers,459,459,459,459,459,459,459,459,459
Churned Best Customers,44,44,44,44,44,44,44,44,44
High Spending New Customers,19,19,19,19,19,19,19,19,19
Lost Cheap Customers,400,400,400,400,400,400,400,400,400
Lost Customers,5,5,5,5,5,5,5,5,5
Lowest Spending Active Loyal Customers,1,1,1,1,1,1,1,1,1
Loyal customers and big spenders,185,185,185,185,185,185,185,185,185
Other Loyal Customers,156,156,156,156,156,156,156,156,156


In [90]:
segmented_rfm.to_csv('RFM_analysis.csv')